In [1]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
import torch

In [2]:
torch.manual_seed(42)

In [3]:
torch.manual_seed(42)
n = 100
t = np.linspace(0, 1, n)
elevation_true = 0.8 * ((np.sin(50*t) + np.sin(10*t))/2)+ 0.1 * t   # Profilo di elevazione reale
elevation_measured = elevation_true + 0.1 * torch.rand(n).numpy()  # Dati misurati con rumore

In [4]:
# Operatori differenziali discreti
D = np.eye(n) - np.eye(n, k=-1)
D = D[:-1, :]  # Prima derivata discreta

D2 = np.eye(n) - 2 * np.eye(n, k=1) + np.eye(n, k=2)
D2 = D2[:-2, :]  # Seconda derivata discreta

In [5]:
ds = []

In [6]:
for i in range(1000):

	# Simulazione dei dati


	# Variabile di ottimizzazione
	x = cp.Variable(n)



	# Parametri del problema
	rho_param = np.random.rand()  # Peso della regolarizzazione sulla pendenza
	kappa_max = np.random.rand()   # Limite sulla curvatura

	# Funzione obiettivo
	objective = cp.Minimize(cp.sum_squares(x - elevation_measured))

	# Vincoli
	constraints = []

	# Vincoli sulla curvatura
	constraints += [cp.sum_squares(D2 @ x) <= kappa_max*np.linalg.norm(D2@elevation_measured)**2]
	constraints += [cp.sum_squares(D @ x) <= rho_param*np.linalg.norm(D@elevation_measured)**2]

	# Vincoli sui limiti di elevazione (se noti)
	elevation_min = np.min(elevation_measured) - 0.1
	elevation_max = np.max(elevation_measured) + 0.1
	#constraints += [x >= elevation_min]
	#constraints += [x <= elevation_max]

	# Risoluzione del problema
	prob = cp.Problem(objective, constraints)
	prob.solve()

	# Soluzione ottimale
	x_hat = x.value

	if prob.status == 'optimal': ds.append([np.array(rho_param), np.array(kappa_max), x.value, constraints[0].dual_value, constraints[1].dual_value])

In [7]:
(x.value).shape

(100,)

In [8]:
constraints[0].dual_value

array([1.26120522e-07])

In [9]:
np.concatenate([x.value, constraints[0].dual_value]).shape

(101,)

In [10]:
constraints[1]

Inequality(Expression(CONVEX, NONNEGATIVE, ()))

In [11]:
constraints[1].dual_value

array([9.02101509])

In [12]:
constraints[0].dual_value

array([1.26120522e-07])

In [13]:
np.linalg.norm(D2@x.value)**2 - kappa_max*np.linalg.norm(D2@elevation_measured)**2

-0.022150712461295526

In [14]:
np.linalg.norm(D@x.value)**2 - rho_param*np.linalg.norm(D@elevation_measured)**2

-2.0166063263715728e-10

In [15]:
D.shape

(99, 100)

In [16]:
np.linalg.norm(2*(x.value - elevation_measured) + 2*constraints[1].dual_value*(D.T@(D@x.value))+ 2*constraints[0].dual_value*(D2.T@(D2@x.value)))

0.00018934503044961822

In [17]:
constraints[1].dual_value*(np.linalg.norm(D@x.value)**2 - rho_param*np.linalg.norm(D@elevation_measured)**2)

array([-1.81918361e-09])

In [18]:
constraints[0].dual_value*(np.linalg.norm(D2@x.value)**2 - kappa_max*np.linalg.norm(D2@elevation_measured)**2)

array([-2.79365943e-09])

In [19]:
ds

[[array(0.97128742),
  array(0.69961994),
  array([ 0.09135021,  0.32052961,  0.46845864,  0.60625204,  0.56728796,
          0.48296357,  0.31125002,  0.19062505,  0.07083784, -0.04144731,
          0.0547748 ,  0.17015872,  0.38040473,  0.57250363,  0.76102463,
          0.84830396,  0.88681357,  0.76738316,  0.56905828,  0.38311327,
          0.1625306 ,  0.033388  , -0.01942279,  0.06303419,  0.13395548,
          0.30571527,  0.47044293,  0.56320365,  0.59650254,  0.47646901,
          0.36365953,  0.0793323 , -0.10017281, -0.31037013, -0.43821121,
         -0.41180027, -0.33681997, -0.16671582, -0.0276897 ,  0.0903096 ,
          0.17446896,  0.12911851,  0.03648961, -0.12895164, -0.35686275,
         -0.58045549, -0.65816352, -0.68142521, -0.58833024, -0.46703868,
         -0.27570822, -0.05243569,  0.09091261,  0.21335993,  0.16453538,
          0.04942961, -0.11693968, -0.2842715 , -0.36696694, -0.44140102,
         -0.34064595, -0.19104836,  0.07119395,  0.2910689 ,  0.510798

In [20]:
from pickle import dump
dump(ds, open("smooth.pkl", "wb"))

In [21]:
x.value.shape

(100,)

In [22]:
D2.shape

(98, 100)

In [23]:
np.linalg.norm(D2@elevation_measured)**2

1.084829414601063

In [24]:
np.linalg.norm(D2 @ elevation_true)**2

0.49741764541027556

In [25]:
np.max(D2 @ elevation_true)

0.10242222754502994

In [26]:
np.linalg.norm(D2 @ x.value)

0.21989909345957004